# 1. Get Data
[Source](https://www.kaggle.com/robroseknows/fetch-google-safe-browsing-transparency-report)

In [ ]:
# This fetches the data displayed on the Google Safe Browsing Transparency Report and outputs it as a CSV
# that can be imported into a Kaggle dataset.
# The original visualization can be found here: https://transparencyreport.google.com/safe-browsing/overview

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from datetime import datetime, date
RUN_TIME = int(datetime.utcnow().timestamp() * 1000)
START_TIME = datetime.fromtimestamp(1148194800000 // 1000)

# Here are the URL requests I found on the page: 
UNSAFE_URL = f"https://transparencyreport.google.com/transparencyreport/api/v3/safebrowsing/sites?dataset=0&series=malwareDetected,phishingDetected&start=1148194800000&end={RUN_TIME}"
NUMBER_URL = f"https://transparencyreport.google.com/transparencyreport/api/v3/safebrowsing/sites?dataset=1&series=malware,phishing&start=1148194800000&end={RUN_TIME}"
SITES_URL = f"https://transparencyreport.google.com/transparencyreport/api/v3/safebrowsing/sites?start=1148194800000&series=attack,compromised&end={RUN_TIME}"
BROWSER_WARNINGS_URL = f"https://transparencyreport.google.com/transparencyreport/api/v3/safebrowsing/warnings?dataset=users&start=1148194800000&end={RUN_TIME}&series=users"
SEARCH_WARNINGS_URL = f"https://transparencyreport.google.com/transparencyreport/api/v3/safebrowsing/warnings?dataset=search&start=1148194800000&end={RUN_TIME}&series=search"
RESPONSE_TIME_URL = f"https://transparencyreport.google.com/transparencyreport/api/v3/safebrowsing/notify?dataset=1&start=1148194800000&end={RUN_TIME}&series=response"
REINFECTION_URL = f"https://transparencyreport.google.com/transparencyreport/api/v3/safebrowsing/notify?dataset=0&start=1148194800000&end={RUN_TIME}&series=reinfect"

COLUMN_NAMES = [
    "WeekOf",
    "Malware sites detected",
    "Phishing sites detected",
    "Malware sites number",
    "Phishing sites number",
    "Attack sites",
    "Compromised sites",
    "Browser warnings",
    "Search warnings",
    "Webmaster response time",
    "Reinfection rate"
    ]

def load_dataframe():
    dates = pd.date_range(start=START_TIME, end=datetime.fromtimestamp(RUN_TIME // 1000), freq='W', normalize=True)
    df = pd.DataFrame(columns=COLUMN_NAMES)
    df["WeekOf"] = dates
    df = df.set_index("WeekOf")
    return df
    
df = load_dataframe()

import requests
import json
def fetch_as_json(url):
    r = requests.get(url)
    c = r.content
    c = c[5:]
    j = json.loads(c)
    return j[0][1]

def malware_phishing_detected(df):
    pts = fetch_as_json(UNSAFE_URL)
    for pt in pts:
        date = pd.to_datetime(pt[0], unit='ms').normalize()
        malware = pt[1][0]
        phishing = pt[1][1]
        malware = malware[0] if malware else np.NaN
        phishing = phishing[0] if phishing else np.NaN
        df[COLUMN_NAMES[1]][date] = malware
        df[COLUMN_NAMES[2]][date] = phishing
    return df

def malware_phishing_number(df):
    pts = fetch_as_json(NUMBER_URL)
    for pt in pts:
        date = pd.to_datetime(pt[0], unit='ms').normalize()
        malware = pt[1][0]
        phishing = pt[1][1]
        malware = malware[0] if malware else np.NaN
        phishing = phishing[0] if phishing else np.NaN
        df[COLUMN_NAMES[3]][date] = malware
        df[COLUMN_NAMES[4]][date] = phishing
    return df
        
def site_count(df):
    pts = fetch_as_json(SITES_URL)
    for pt in pts:
        date = pd.to_datetime(pt[0], unit='ms').normalize()
        attack = pt[1][0]
        comped = pt[1][1]
        attack = attack[0] if attack else np.NaN
        comped = comped[0] if comped else np.NaN
        df[COLUMN_NAMES[5]][date] = attack
        df[COLUMN_NAMES[6]][date] = comped
    return df
    
def browser_warnings(df):
    pts = fetch_as_json(BROWSER_WARNINGS_URL)
    for pt in pts:
        date = pd.to_datetime(pt[0], unit='ms').normalize()
        value = pt[1][0]
        value = value[0] if value else np.NaN
        df[COLUMN_NAMES[7]][date] = value
    return df
    
def search_warnings(df):
    pts = fetch_as_json(SEARCH_WARNINGS_URL)
    for pt in pts:
        date = pd.to_datetime(pt[0], unit='ms').normalize()
        value = pt[1][0]
        value = value[0] if value else np.NaN
        df[COLUMN_NAMES[8]][date] = value
    return df
    
def response_time(df):
    pts = fetch_as_json(RESPONSE_TIME_URL)
    for pt in pts:
        date = pd.to_datetime(pt[0], unit='ms').normalize()
        value = pt[1][0]
        value = value[0] if value else np.NaN
        df[COLUMN_NAMES[9]][date] = value
    return df
    
def reinfection_rate(df):
    pts = fetch_as_json(REINFECTION_URL)
    for pt in pts:
        date = pd.to_datetime(pt[0], unit='ms').normalize()
        value = pt[1][0]
        # Multiply by 100 and cast to int to save space on import.
        value = int(value[1] * 100) if value else np.NaN
        df[COLUMN_NAMES[10]][date] = value
    return df

df = malware_phishing_detected(df)
df = malware_phishing_number(df)
df = site_count(df)
df = browser_warnings(df)
df = search_warnings(df)
df = response_time(df)
df = reinfection_rate(df)
df.to_csv("data.csv", header=True, index=True, index_label="WeekOf")

df.tail(3)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#Let's make this notebook reproducible 
np.random.seed(42)

import random
random.seed(42)

import warnings
warnings.filterwarnings('ignore')

# Make the default figures a bit bigger
plt.rcParams['figure.figsize'] = (7,4.5) 
plt.rcParams["figure.dpi"] = 140 

sns.set(style="ticks")
sns.set_context("poster", font_scale = .7, rc={"grid.linewidth": 5})

greek_salad = ['#D0D3C5', '#56B1BF', '#08708A', '#D73A31', '#032B2F']
sns.set_palette(greek_salad)

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
from plotly.graph_objs import *
import plotly.graph_objs as go

# 2. Insights

**Malware** <br> These websites contain code that installs *malicious software* onto visitors’ computers, <br> either when a user thinks they are downloading legitimate software or without a user’s knowledge.

**Phishing** <br> These websites *pretend to be legitimate* so that they can trick <br> users into typing in their usernames and passwords or sharing other private information.

## Unsafe websites detected per week

In [ ]:
# Unsafe websites detected per week
trace0 = go.Scatter(
    x = df.index,
    y = df['Malware sites detected'],
    name = 'Malware sites',
    line = dict(
        shape='spline',
        color = (greek_salad[2]),
        width = 2),
    connectgaps=True
)
trace1 = go.Scatter(
    x = df.index,
    y = df['Phishing sites detected'],
    name = 'Phishing sites',
    line = dict(
        shape='spline',
        color = (greek_salad[3]),
        width = 2),
    connectgaps=True
)

data = [trace0, trace1]

layout = dict(
    title = "Unsafe websites detected per week",
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=3,
                     label='3m',
                     step='month',
                     stepmode='backward'),
                dict(count=12,
                     label='1y',
                     step='month',
                     stepmode='backward'),
                dict(count=36,
                     label='3y',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = False
        ),
        type='date',
        showgrid=False
    ),
    yaxis=dict(
        showgrid=False
    ),
    legend=dict(
        y=0.5,
        traceorder='reversed',
        font=dict(
            size=16
        )
    )
)

fig = dict(data=data, layout=layout)
iplot(fig, filename='basic-line')

## Websites deemed dangerous by Safe Browsing

In [ ]:
# Websites deemed dangerous by Safe Browsing
trace0 = go.Scatter(
    x = df.index,
    y = df['Malware sites number'],
    name = 'Malware sites',
    line = dict(
        shape='spline',
        color = (greek_salad[2]),
        width = 2),
    connectgaps=True
)
trace1 = go.Scatter(
    x = df.index,
    y = df['Phishing sites number'],
    name = 'Phishing sites',
    line = dict(
        shape='spline',
        color = (greek_salad[3]),
        width = 2),
    connectgaps=True
)

data = [trace0, trace1]

layout = dict(
    title = "Websites deemed dangerous by Safe Browsing",
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=3,
                     label='3m',
                     step='month',
                     stepmode='backward'),
                dict(count=12,
                     label='1y',
                     step='month',
                     stepmode='backward'),
                dict(count=36,
                     label='3y',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = False
        ),
        type='date',
        showgrid=False
    ),
    yaxis=dict(
        showgrid=False
    ),
    legend=dict(
        y=0.5,
        traceorder='reversed',
        font=dict(
            size=16
        )
    )
)

fig = dict(data=data, layout=layout)
iplot(fig, filename='basic-line')

## Sites hosting malware detected per week

**Attack sites** <br> These are websites that hackers have set up to *intentionally* host and distribute malicious software. <br> These sites directly exploit a browser or contain harmful software that often exhibit malicious behaviors. 

**Compromised sites** <br> These are legitimate websites that have been *hacked* to include content from, <br> or to direct users to, sites that may exploit their browsers. 




In [ ]:
# Websites deemed dangerous by Safe Browsing
trace0 = go.Scatter(
    x = df.index,
    y = df['Attack sites'],
    name = 'Attack sites',
    line = dict(
        shape='spline',
        color = (greek_salad[2]),
        width = 2),
    connectgaps=True,
    yaxis='y2'
)
trace1 = go.Scatter(
    x = df.index,
    y = df['Compromised sites'],
    name = 'Compromised sites',
    line = dict(
        shape='spline',
        color = (greek_salad[3]),
        width = 2),
    connectgaps=True
)

data = [trace0, trace1]

layout = dict(
    title = "Sites hosting malware detected per week",
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=3,
                     label='3m',
                     step='month',
                     stepmode='backward'),
                dict(count=12,
                     label='1y',
                     step='month',
                     stepmode='backward'),
                dict(count=36,
                     label='3y',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = False
        ),
        type='date',
        showgrid=False
    ),
    yaxis=dict(
        title='Compromised sites',
        titlefont=dict(
            color=greek_salad[3],
            size=16
        ),
        tickfont=dict(
            color=greek_salad[3]
        ),
        showgrid=False
    ),
    yaxis2=dict(
        title='Attack sites',
        titlefont=dict(
            color=greek_salad[2],
            size=16
        ),
        tickfont=dict(
            color=greek_salad[2]
        ),
        overlaying='y',
        side='right',
        showgrid=False
    ),
    showlegend=False
)

fig = dict(data=data, layout=layout)
iplot(fig, filename='basic-line')

## Warning displayed to users

These warnings are designed to prevent users from visiting harmful sites and help them stay safe online.

In [ ]:
# Websites deemed dangerous by Safe Browsing
trace0 = go.Scatter(
    x = df.index,
    y = df['Search warnings'],
    name = 'Search warnings',
    line = dict(
        shape='spline',
        color = (greek_salad[2]),
        width = 2),
    connectgaps=True,
    yaxis='y2'
)
trace1 = go.Scatter(
    x = df.index,
    y = df['Browser warnings'],
    name = 'Browser warnings',
    line = dict(
        shape='spline',
        color = (greek_salad[3]),
        width = 2),
    connectgaps=True
)

data = [trace0, trace1]

layout = dict(
    title = "Warnings displayed to users",
    xaxis=dict(
        range = ['2011-01-01',df.index.max()],
        rangeselector=dict(
            buttons=list([
                dict(count=3,
                     label='3m',
                     step='month',
                     stepmode='backward'),
                dict(count=12,
                     label='1y',
                     step='month',
                     stepmode='backward'),
                dict(count=36,
                     label='3y',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = False
        ),
        type='date',
        showgrid=False
    ),
    yaxis=dict(
        title='Browser warnings',
        titlefont=dict(
            color=greek_salad[3],
            size=16
        ),
        tickfont=dict(
            color=greek_salad[3]
        ),
        showgrid=False
    ),
    yaxis2=dict(
        title='Search warnings',
        titlefont=dict(
            color=greek_salad[2],
            size=16
        ),
        tickfont=dict(
            color=greek_salad[2]
        ),
        overlaying='y',
        side='right',
        showgrid=False
    ),
    showlegend=False
)

fig = dict(data=data, layout=layout)
iplot(fig, filename='basic-line')

## Webmaster response time 
How quickly webmasters clean up their sites after receiving notifications that their sites have been compromised.

In [ ]:
# Websites deemed dangerous by Safe Browsing
trace0 = go.Scatter(
    x = df.index,
    y = df['Webmaster response time'],
    name = 'days',
    line = dict(
        shape='spline',
        color = (greek_salad[2]),
        width = 2),
    connectgaps=True,
)

data = [trace0]

layout = dict(
    title = "Webmaster response time",
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=3,
                     label='3m',
                     step='month',
                     stepmode='backward'),
                dict(count=12,
                     label='1y',
                     step='month',
                     stepmode='backward'),
                dict(count=36,
                     label='3y',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = False
        ),
        type='date',
        showgrid=False
    ),
    yaxis=dict(
        title='Response time in days',
        titlefont=dict(
            color=greek_salad[2],
            size=16
        ),
        tickfont=dict(
            color=greek_salad[2]
        ),
        showgrid=False
    ),
    showlegend=False
)

fig = dict(data=data, layout=layout)
iplot(fig, filename='basic-line')

## Reinfection rate

In [ ]:
# Websites deemed dangerous by Safe Browsing
trace0 = go.Scatter(
    x = df.index,
    y = df['Reinfection rate'],
    name = 'days',
    line = dict(
        shape='spline',
        color = (greek_salad[3]),
        width = 2),
    connectgaps=True,
)

data = [trace0]

layout = dict(
    title = "Reinfection rate",
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=3,
                     label='3m',
                     step='month',
                     stepmode='backward'),
                dict(count=12,
                     label='1y',
                     step='month',
                     stepmode='backward'),
                dict(count=36,
                     label='3y',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = False
        ),
        type='date',
        showgrid=False
    ),
    yaxis=dict(
        title='Reinfection rate in percentage',
        titlefont=dict(
            color=greek_salad[3],
            size=16
        ),
        tickfont=dict(
            color=greek_salad[3]
        ),
        showgrid=False
    ),
    showlegend=False
)

fig = dict(data=data, layout=layout)
iplot(fig, filename='basic-line')